In [1]:
!pip install simple-lama-inpainting
!apt-get install -y libgl1
!pip install ultralytics
!pip install kagglehub

  Using cached simple_lama_inpainting-0.1.2-py3-none-any.whl.metadata (1.9 kB)
  Using cached fire-0.5.0.tar.gz (88 kB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached Pillow-9.5.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.5 kB)
  Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64

In [2]:
from ultralytics import YOLO
from simple_lama_inpainting import SimpleLama
from PIL import Image
import cv2
import numpy as np
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
model = YOLO("yolo11n-seg.pt")
simple_lama = SimpleLama()

Downloading: "https://github.com/enesmsahin/simple-lama-inpainting/releases/download/v0.1.0/big-lama.pt" to /root/.cache/torch/hub/checkpoints/big-lama.pt
100%|██████████| 196M/196M [00:06<00:00, 33.0MB/s] 


In [4]:
images_dir= os.path.join('awsaf49','coco-2017-dataset','versions','2','coco2017','train2017')
images_dict=dict()

In [5]:
def generate_masks_and_target_image(model,image_path,padding=10):
    image_arr = cv2.imread(image_path)
    image_name = image_path.split("/")[-1]
    image_name = image_name.replace(".jpg","")
    results = model(image_arr)
    images_dict[image_name]=dict()
    images_dict[image_name]["masks"] = list()
    images_dict[image_name]["targets"] = list()
    image_area = image_arr.shape[0] * image_arr.shape[1]
    if  len(results[0].boxes)>3:
        return
    for i, (xmin, ymin, xmax, ymax) in enumerate(results[0].boxes.xyxy.cpu().numpy()):
        xmin_p = max(0, int(xmin) - padding)
        ymin_p = max(0, int(ymin) - padding)
        xmax_p = min(image_arr.shape[1], int(xmax) + padding)
        ymax_p = min(image_arr.shape[0], int(ymax) + padding)
        mask_area = (xmax_p - xmin_p) * (ymax_p - ymin_p)
        percent_area = mask_area / image_area
        if percent_area > 0.6:
            continue
        mask = np.zeros(image_arr.shape[:2], dtype=np.uint8)
        mask[ymin_p:ymax_p, xmin_p:xmax_p] = 255
        mask_filename = os.path.join("masks",image_name+"_mask_"+str(i)+".png")
        target_filename = os.path.join("targets",image_name+"_target_"+str(i)+".png")
        target_image = image_arr[ymin_p:ymax_p, xmin_p:xmax_p]
        images_dict[image_name]["masks"].append(mask_filename)
        images_dict[image_name]["targets"].append(target_filename)
        cv2.imwrite(mask_filename, mask)
        cv2.imwrite(target_filename, target_image)

def generate_target_and_background_image(lama_model,image_path,mask_path,save_path):
    image = Image.open(image_path)
    mask = Image.open(mask_path)
    result = lama_model(image,mask)
    result.save(save_path)
    

In [6]:
for file in os.listdir(images_dir)[:1000]:
    image_path = os.path.join(images_dir,file)
    generate_masks_and_target_image(model,image_path)


0: 480x640 3 bowls, 1 broccoli, 200.4ms
Speed: 10.5ms preprocess, 200.4ms inference, 285.4ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 2 giraffes, 175.8ms
Speed: 7.2ms preprocess, 175.8ms inference, 4.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 potted plant, 1 dining table, 1 vase, 31.8ms
Speed: 4.2ms preprocess, 31.8ms inference, 4.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 zebra, 29.4ms
Speed: 5.0ms preprocess, 29.4ms inference, 5.5ms postprocess per image at shape (1, 3, 448, 640)

0: 640x512 1 person, 1 umbrella, 162.6ms
Speed: 5.0ms preprocess, 162.6ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 512)

0: 480x640 1 dog, 63.5ms
Speed: 6.1ms preprocess, 63.5ms inference, 5.2ms postprocess per image at shape (1, 3, 480, 640)

0: 640x512 4 persons, 2 horses, 1 potted plant, 75.0ms
Speed: 8.2ms preprocess, 75.0ms inference, 7.6ms postprocess per image at shape (1, 3, 640, 512)

0: 512x640 2 persons, 3 elepha

In [7]:
images_dict

{'000000000009': {'masks': [], 'targets': []},
 '000000000025': {'masks': ['masks/000000000025_mask_0.png',
   'masks/000000000025_mask_1.png'],
  'targets': ['targets/000000000025_target_0.png',
   'targets/000000000025_target_1.png']},
 '000000000030': {'masks': ['masks/000000000030_mask_0.png',
   'masks/000000000030_mask_1.png',
   'masks/000000000030_mask_2.png'],
  'targets': ['targets/000000000030_target_0.png',
   'targets/000000000030_target_1.png',
   'targets/000000000030_target_2.png']},
 '000000000034': {'masks': [], 'targets': []},
 '000000000036': {'masks': ['masks/000000000036_mask_1.png'],
  'targets': ['targets/000000000036_target_1.png']},
 '000000000042': {'masks': ['masks/000000000042_mask_0.png'],
  'targets': ['targets/000000000042_target_0.png']},
 '000000000049': {'masks': [], 'targets': []},
 '000000000061': {'masks': [], 'targets': []},
 '000000000064': {'masks': [], 'targets': []},
 '000000000071': {'masks': ['masks/000000000071_mask_0.png'],
  'targets': ['

In [8]:
for key in images_dict.keys():
    print(key)
    masks = images_dict[key]["masks"]
    original_image_path = os.path.join(images_dir,key+".jpg")
    images_dict[key]["backgrounds"] = list()
    for i,mask in enumerate(masks):
        background_file_name = os.path.join("backgrounds",key+"_background_"+str(i)+".png")
        try:
            generate_target_and_background_image(simple_lama,original_image_path,mask,background_file_name)
        except:
            print(f'Error:{key}')
            continue
        images_dict[key]["backgrounds"].append(background_file_name)
    
    

000000000009
000000000025
000000000030
000000000034
000000000036
000000000042
000000000049
000000000061
000000000064
000000000071
000000000072
000000000073
000000000074
000000000077
000000000078
000000000081
000000000086
Error:000000000086
Error:000000000086
000000000089
000000000092
000000000094
000000000109
000000000110
000000000113
000000000127
000000000133
000000000136
000000000138
000000000142
000000000143
000000000144
000000000149
000000000151
000000000154
000000000164
000000000165
000000000192
000000000194
000000000196
000000000201
000000000208
000000000241
000000000247
000000000250
000000000257
000000000260
000000000263
000000000283
000000000294
000000000307
000000000308
000000000309
000000000312
000000000315
000000000321
000000000322
000000000326
000000000328
000000000332
000000000338
000000000349
000000000357
000000000359
000000000360
000000000368
000000000370
000000000382
000000000384
000000000387
000000000389
000000000394
000000000395
000000000397
000000000400
000000000404


In [9]:
images_dict

{'000000000009': {'masks': [], 'targets': [], 'backgrounds': []},
 '000000000025': {'masks': ['masks/000000000025_mask_0.png',
   'masks/000000000025_mask_1.png'],
  'targets': ['targets/000000000025_target_0.png',
   'targets/000000000025_target_1.png'],
  'backgrounds': ['backgrounds/000000000025_background_0.png',
   'backgrounds/000000000025_background_1.png']},
 '000000000030': {'masks': ['masks/000000000030_mask_0.png',
   'masks/000000000030_mask_1.png',
   'masks/000000000030_mask_2.png'],
  'targets': ['targets/000000000030_target_0.png',
   'targets/000000000030_target_1.png',
   'targets/000000000030_target_2.png'],
  'backgrounds': ['backgrounds/000000000030_background_0.png',
   'backgrounds/000000000030_background_1.png',
   'backgrounds/000000000030_background_2.png']},
 '000000000034': {'masks': [], 'targets': [], 'backgrounds': []},
 '000000000036': {'masks': ['masks/000000000036_mask_1.png'],
  'targets': ['targets/000000000036_target_1.png'],
  'backgrounds': ['backg

In [10]:
import pickle 

with open('images_dict.pkl', 'wb') as f:
    pickle.dump(images_dict, f)
        